In [76]:
import requests
import base64
import json
import logging
import os
import sys
import pytz
import warnings
import gc
import pandas as pd
from pathlib import Path
from datetime import datetime

warnings.filterwarnings("ignore")

base_url = 'https://jira.footlocker.com/rest/api/2/search'

In [77]:

discovery_state = ["created",
    "New",
    "Ready for Development",
    "PO Validation",
    "PO Grooming",
    "Ready for Refinement",
    "To Do",
    "FA Grooming",
    "Open",
    "In Planning",
    "Dev Grooming/Estimation",
    "Technical Grooming",
    "Deferred",
    "Backlog",
    "Scrum of Scrums Items",
    "Refinement",
    "Ready for Assignment",
    "Selected for Development",
    "M Editorial Request Drafts",
    "Pending Evaluation",
    "Add to Backlog",
    "Prioritized",
    "Intake",
    "Request Acknowledged",
    "Design",
    "Intake Request",
    "Initiated",
    "Assigned",
    "Triage",
    "Functional Grooming",
    "Style Review",
    "Acceptance / Review",
    "Hold for Dependencies",
    "Awaiting Requirements",
    "Action Needed",
    "On-Hold",
    "Ready For Deployment",
    "Grooming",
    "Analysis in Progress",
    "Waiting for Information",
    "M Editorial To Do",
    "M Editorial In Progress",
    # "S&D Request Drafts",
    # "S&D To Do",
    "SM Backlog",
    "Discovery Gate Prep",
    "CAR Submitted",
    "Ready for Requirements",
    "Req-Arch in Progress",
    "Ready for Estimation",
    "Ready for Planning",
    "Ready for Review",
    "Planning in Progress",
    "Architect Review",
    "New Project Requests",
    "Approved Project Backlog",
    "Discovery",
    "Architect Review Dependencies",
    "JDAP Ready",
    "Cleaned Up (Temporary)",
    "With Vendor",
    # "S&D Stakeholder Review",
    "Ready for test",
    "First Priority",
    "Results Ready",
    "Analysis",
    "ToDo",
    "Ready",
    "Integration Test",
    # "Approved & Prioritized",
    "Discussion/Review",
    "Pending",
    "Ready to Work",
    "Functional",
    "NEEDS MORE INFO",
    "IN UX/UI",
    "Needs Approval",
    "Assessing",
    "Submitted for Assessment",
    "Pending Decision",
    "Decided",
    "Ready for Dev",
    "Action Required",
    "Submitted",
    "Planning",
    "Measurement Requirements",
    "UX Design",
    "Stakeholder Review",
    "Stakeholder Kickoff",
    "Research",
    "Analyze Results",
    "Reviewed",
    "Requirements",
    "On Hold - To Do",
    "Ice Box",
    "Ready for Deploy",
    "TO-DO",
    "On Hold/Ready",
    "Reseach",
    "Wireframes To Do",
    "New Requests",
    "Approved for Estimation",
    "Estimated / Ready for Review",
    "Budgeted",
    "Tech Feasibility",
    "Product Review",
    "Ready For Approval",
    "New Request",
    "Defect Created",
    "Documentation and Tie-off",
    "Business Case Review",
    "Proposed",
    "Under Evaluation",
    "Under Review",
    "Request Drafts",
    "Parked for dependency",
    "Parked",
    "On Hold by FL",
    "Begin",
    "Concept",
    "Change in Requirements",
    "Ready For SNUG Review",
    "Waiting Approval",
    "Pending Acceptance Criteria",
    "Wishlist",
    "Approved Projects",
    "Epics",
    "Delayed",
    "Not Started",
    "Drafting",
    "Pending PO Review",
    "PO Acceptance",
    "Rework",
    "Ready for Implementation",
    "SE Backlog",
    "In UX/UI Review",
    "Planning / Requirements",
    # "Ready for Dev Grooming & Estimation",
    "Scoped",
    "Ready to Diagnose",
    "Waiting Feedback",
    "To be reviewed",
    "Board Review",
    "Business Review",
    "Wireframe IP",
    "Mockup To Do",
    "Wire Frame Needs Revision",
    "Estimation",
    "In Progress - UI",
    "Priority List",
    "Priority List - Ready for UI",
    "UX",
    "UX - Prioritized",
    "PO - Need More Information",
    "UX - In Progress",
    "UX - On Hold (Waiting for Peer Review)",
    "UX - Progress",
    "PO - On Hold",
    "Copy - In Progress",
    "UI - Prioritized",
    "PO - Unprioritized",
    "Copy - Prioritized",
    "UI-In Progress",
    "UX - On Hold (Waiting for Results)",
    "UX - On Hold (Waiting for Deploy)",
    "UI - Ready for Style Review",
    "In Progress - UX",
    "Needs Approval - PO",
    "UI - Needs Approval",
    "On Hold - PO",
    "Handed Off - Awaiting Dev",
    "Prioritized - UXD",
    "Design Idea",
    "Product Owner / Stakeholder Review",
    "Research Idea",
    "Verification",
    "WorkflowState"]


cancel_state = ["Closed - Not Doing",
    "Duplicate",
    "Closed - No Deploy",
    "Denied",
    "Canceled",
    "Rejected",
    "Cancel",
    "Cancelled",
    "Not Approved / Out of Scope",
    "Invalid",
    "Failed",
    "Closed - Inactivity",
    "Closed - Duplicate"]

close_state = ["Closed",
    "Complete",
    "Done",
    "Resolved",
    "Pass",
    # "Fixed",
    "Closed - Pending Review",
    "Archived",
    "Archive",
    "Completed",
    # "Completed - Change(s) Made",
    # "Closed - Archive"
    ]

development_state = ["In Development",
    "Code Review",
    "On Hold",
    "In Progress",
    "Dev Blocked",
    "Deploy to Staging",
    "Dev Review",
    "Blocked",
    "Dev - In Progress",
    "Dev Complete",
    "Ready for Stage",
    "Reopened",
    "Build",
    "In Review",
    "Testing",
    "Review",
    "Work in Progress",
    "Dev In Progress",
    "Dev Complete-Pull Request",
    "Development",
    # "S&D In Progress",
    # "S&D Blocked",
    "Solution Review",
    "Peer Review",
    "Workaround",
    "Needs Review",
    "UA Review",
    "Merge Request",
    "Ready for Code Review",
    "Approval Needed",
    "PO Approval",
    "Reopen",
    "Ready for PR",
    "Sprint Test",
    "Demo",
    # "Remedy In Progress",
    "In Design",
    # "Staged",
    # "Development in Progress",
    "Merged",
    "Reivew",
    "In Code Review",
    # "Needs Style Review",
    # "Develop",
    # "Working",
    # "Style/Code Review",
    "Dev Done",
    # "Pull Request Review",
    "Executing",
    "Needs Revision",
    "Active",
    # "Wireframe Review",
    # "Mockup IP",
    # "Mockup Review",
    # "PO - Needs Approval"
    ]

deployment_state = ["Monitoring",
    "Production Review",
    "Ready for Prod",
    "Deployed",
    "Deploy",
    "Validate/Verify",
    # "S&D Complete",
    "Deployed to Production",
    "Deploying Increments",
    "Approved",
    "Validation",
    "Ready to Deploy",
    "Reporter Approval",
    # "Implemented",
    "Production",
    "CR Submitted",
    "Deploy to Production",
    "In Production",
    "Post-Production",
    "Validation Complete",
    # "Deployed to Prod",
    # "Staged for Deploy",
    # "Released Live",
    "Not a Bug",
    "Ready for Production", 
    "Deploy to Prod"]

qa_state =["QA Testing",
    "Future Test Cases",
    "Deploy to Test",
    "Deploy to UAT",
    "QA (UAT)",
    "QA Blocked",
    "QA (Test)",
    "Dev Complete / Deploy to Test",
    "QA (Staging)",
    "QA (Prod)",
    "UA (Test)",
    "SIT",
    "Ready for QA",
    "QA In Progress",
    "Ready for UAT",
    "QA Staging",
    "In QA",
    "UAT - In Progress",
    "In Testing",
    "QA",
    "UAT",
    "Test",
    "Testing / Integration (Test)",
    "Deploy to QA",
    "QA - In Progress",
    "UAT tesing",
    "test1",
    "Testing in UAT",
    "Ready for Testing",
    "UAT Validation",
    "UAT - Testing",
    "Dev QA",
    # "QA Automation",
    # "QA Icebox",
    # "Quality Assurance",
    "Deployed to UAT",
    "User Acceptance Testing",
    "UAT - In Review",
    "To Be Tested",
    "Sandbox",
    "UAT - Complete",
    # "Stores UAT",
    "QA / Testing"]



In [78]:

def get_report(base_url,api_token):
    logger.info("Getting report results...")
    header_gs = {'Accept': 'application/json'}
    headers = { 'Authorization' : 'Bearer %s' %  api_token}
    # 'Content-Length':3000 }
    logger.info(base_url)
    
    for i in range(0,3):
        r = requests.get(base_url,headers=headers)
        logger.info(r)
        if r.ok:
            logger.info("Report results received...")
            logger.info("HTTP %i - %s" % (r.status_code, r.reason))
            return r.text
        else:
            logger.error("HTTP %i - %s" % (r.status_code, r.reason))
            continue
    return None

In [79]:
def getDataSet(query, project, issueType,api_token):
    pd_obj = None
    maxRslt = 500
    totalRsltFetch = -1
    df= pd.DataFrame()
    while True:
        dataset= get_report(base_url+query+f"&maxResults={maxRslt}&startAt={totalRsltFetch + 1 }", api_token)
        if dataset != None:
            obj = json.loads(dataset)
            logger.info(f"total: {obj['total']} ({project} - {issueType})")
            totalRsltFetch = totalRsltFetch + maxRslt

            if obj['issues']:
                df = pd.concat([df, pd.json_normalize(obj['issues']) ], ignore_index=True)
            else:
               logger.error(f'No records found ({project} - {issueType})....')
            if (totalRsltFetch >= obj['total'] ):
                break
        # pd_obj = pd.json_normalize(obj['issues'])
        else:
            logger.error(f'Dataset was empty ({project} - {issueType})....')
            break
        # break       

    return df
    

In [80]:
def format_timedelta(td):
    """Formats a timedelta object to 'day.hours:min:sec' format."""

    seconds = td.total_seconds()
    days, seconds = divmod(seconds, 86400)
    hours, seconds = divmod(seconds, 3600)
    minutes, seconds = divmod(seconds, 60)

    return f"{int(days)}.{int(hours):02}:{int(minutes):02}:{int(seconds):02}"

In [81]:
def convert_to_key_value(string):
    result = {}
   
    if string :
        pairs = string[0].split(',')  # Split string by comma
        # logger.info(pairs)
        idx =1
        for pair in pairs:
            key, value = pair.split('=')  # Split each pair by '='
            result[key.strip()] = value.strip()  # Add to dictionary, stripping whitespace
            idx = idx +1 
            if idx > 9:
                break
    return result

In [82]:
def split_list(lst, n):
    """Splits a list into n approximately equal parts."""
    k, m = divmod(len(lst), n)
    
    return [lst[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n)]

In [83]:
def getUrlDataSet(url,api_token):
    pd_obj = None

    dataset= get_report(url, api_token)
    if dataset != None:
        pd_obj = json.loads(dataset)
        # pd_obj = pd.json_normalize(obj['issues'])
    return pd_obj

In [84]:
def setup_logger( level=logging.WARNING):
    """Configures a logger and returns it."""

    # if not os.path.exists('./log'):
    #     os.makedirs('./log')

    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

    # handler = logging.FileHandler('./log/soleProj.log')
    handler = logging.StreamHandler()
    handler.setFormatter(formatter)
   
    logger = logging.getLogger()
    logger.setLevel(level)
    
    if (logger.hasHandlers()):
        logger.handlers.clear()

    logger.addHandler(handler)
   

    return logger

In [92]:
def projectList(api_token):
    searchURL='https://jira.footlocker.com/rest/api/2/project'
    searchQry_DS = getUrlDataSet(searchURL,api_token)
    df=pd.json_normalize(searchQry_DS)
    df=df.drop(df[df['key'].isin(['DEVOPS','EPL'])].index)
    return df['key']

In [86]:

def sprintvalue(x,field):
    if(len(x)!=0):
      return x[field]
    else:
     return 'NA'

In [87]:
def keyvalue(x,field):
    if(len(x)!=0):
      return x[0][field]
    else:
     return 'NA'


In [88]:
def getBaselineLst(element):
   baselinelst =[]
   if element != None:
      if  element != 'NA':
         for idx in element:
            baselinelst.append(idx['value'])
   return baselinelst

In [89]:
def clean_folder(filepath,finder):
    # Get all .csv files in the 'data' directory
    csv_files = Path(filepath).glob(f"*{finder}")                                   
    for file in csv_files:
        filename=file.name
        newname= filename.replace(finder,'.csv')
        logger.warning(f'renaming {filename} TO {newname}....')
        os.rename(os.path.join(filepath, filename), os.path.join(filepath, newname))
        os.utime(os.path.join(filepath, newname), None)

In [90]:
def append_to_csv(df, csv_file):
    """Appends a DataFrame to a CSV file, handling missing columns."""

    # Check if the file exists and read the header if it does
    try:
        existing_df = pd.read_csv(csv_file, nrows=0)
        header = list(existing_df.columns)
    except FileNotFoundError:
        header = None

    # Reindex the DataFrame to match the existing header (if it exists)
    if header:
        df = df.reindex(columns=header, fill_value="")

    # Append the DataFrame to the CSV file
    df.to_csv(csv_file, mode='a', index=False, header=header is None)